# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column

y = loans_df['credit_ranking']
# Display a sample of y

y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking

X = loans_df.copy().drop(columns ='credit_ranking')

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance

scaler = StandardScaler()

# Fit the scaler to the features training dataset

scaler.fit(X_train)

# Fit the scaler to the features training dataset

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_features = X_train.shape[1]

# Review the number of features
input_features

11

In [10]:
# Define the number of hidden nodes for the first hidden layer

# Define the number of hidden nodes for the second hidden layer

# Define the number of neurons in the output layer

model = Sequential()

model.add(Dense(6, activation='relu', input_shape=(input_features,))) 
model.add(Dense(3, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

C:\Users\rpesl\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Create the Sequential model instance


# Add the first hidden layer


# Add the second hidden layer


# Add the output layer to the model specifying the number of output neurons and activation function

# ALL DONE ABOVE!!

In [12]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 6)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              21 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled,y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4792 - loss: 0.7185   
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.5096 - loss: 0.6972
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.5605 - loss: 0.6786
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - accuracy: 0.5857 - loss: 0.6581
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.6513 - loss: 0.6371
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.6412 - loss: 0.6413
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.6524 - loss: 0.6185
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.6595 - loss: 0.6158
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.6832 - loss: 0.6127
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.7009 - loss: 0.5842
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.7011 - loss: 0.5844
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data

loss, accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results

print(f"Loss on test data: {loss:.4f}")
print(f"Accuracy on test data: {accuracy:.4%}")


13/13 - 0s - 6ms/step - accuracy: 0.7450 - loss: 0.5345
Loss on test data: 0.5345
Accuracy on test data: 74.5000%


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
from tensorflow.keras.models import load_model
from pathlib import Path

In [17]:
# Set the model's file path

file_path = Path(r"C:\Users\rpesl\OneDrive\Documents\GitHub\neural-network-challenge-1\model_name.keras")

# Export your model to a keras file
model.save(file_path)

model = load_model(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [18]:
# Set the model's file path

model = load_model(file_path)

# Load the model to a new object
print("Model loaded")

Model loaded


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions with the test data

predictions = model.predict(X_test_scaled)


# Display a sample of the predictions
predictions[:5]


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step


array([[0.47641215],
       [0.3265254 ],
       [0.6811763 ],
       [0.5785763 ],
       [0.97902066]], dtype=float32)

In [20]:
# Save the predictions to a DataFrame and round the predictions to binary results

binary_predictions = (predictions > 0.5).astype(int)

predictions_df = pd.DataFrame({
    "Predicted_Label": binary_predictions.flatten()
})

predictions_df

,Predicted_Label
0,0
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [21]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, binary_predictions.flatten(), target_names=['Class 0', 'Class 1'])
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.69      0.83      0.75       188
     Class 1       0.82      0.67      0.74       212

    accuracy                           0.74       400
   macro avg       0.75      0.75      0.74       400
weighted avg       0.76      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

In my Point of View, to build an accurate recommendation system, few variables would be necessary:
1/ Student Data: Demographic details, financial background, academic performance, and previous loan history. This helps in assessing the student's financial needs and loan repayment capability.

2/Loan Data: Information about loan terms, interest rates, repayment options, and benefits offered by various providers. This is crucial for matching loans to student profiles based on suitability.

3/ User Feedback: Ratings and reviews on different loan options by students, helping to refine and improve recommendations.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Given the data types, content-based filtering would be appropriate because it can make personalized recommendations by matching loan features with individual student profiles, without needing input from other users.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

1/ Privacy and Data Security: Ensuring the protection of sensitive personal and financial information is critical due to the risk of data breaches.
2/ Bias and Fairness: Avoiding algorithmic bias to ensure fair and equitable loan recommendations across different demographic and financial backgrounds is essential.